In [0]:
import pandas as pd
import numpy as np
import copy
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My\ Drive/Colab\ Notebooks/Data\ Mining\ -\ CSC440

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/Data Mining - CSC440'
/content/drive/My Drive/Colab Notebooks/Data Mining - CSC440


In [0]:
df = pd.read_csv('adult.data', header = None , skipinitialspace = True , na_values = '?')
df_as_matrix = np.asarray(df)

#for i in df_as_matrix:
#  i[12] = str(i[12])

def getkey(item):
  return item[0] 


def check_if_xinlist(x,l):
  for i in l:
    if x == i:
      return True


def make_L1(data , support):
  L1 = []
  support = support * data.shape[0]
  D = {}
  for i in df_as_matrix:
    for j in i:
      if check_if_xinlist(j,D.keys()):
        D[j] = D[j] + 1
      else:
        D[j] = 1
  for k,v in D.items():
    if v>= support:
      L1.append((v,k))
  L1 = sorted(L1 , key = getkey , reverse = True)
  L1 = [j for i,j in L1]
  return L1

def prune_and_sort(data , L1):
  x2 = []
  X = {}
  for i in data:
    x1 = []
    for j in i:
      if j in L1:
        x1.append(j)
    x2.append(x1)
  for i in range(len(L1)):
    X[L1[i]] = i
  for i in range(len(x2)):
    for j in range(len(x2[i])):
      x2[i][j] = (X[x2[i][j]], x2[i][j])
  for i in x2:
    i.sort(key = getkey)
  for i in range(len(x2)):
    x2[i] = [m for n,m in x2[i]]
  return x2
 
class node:

  def __init__(self, name):
    self.name = name
    self.parent = 'Null'
    self.count = 1
    self.children = []

    
  def add_count(self):
    self.count = self.count + 1

class FP_Tree:

  def __init__(self):
    self.name = 'Null'
    self.children = []

def make_tree(A , itemset):
 
  flag = 0
  for i in range(len(A.children)):
    if A.children[i].name == itemset[0]:
      flag = flag + 1
      break
  if flag == 1:
    A.children[i].add_count()
    itemset = itemset[1:]
    if len(itemset) != 0:
      make_tree(A.children[i] , itemset)
  else:
    x = node(itemset[0])
    A.children.append(x)
    x.parent = A
    itemset = itemset[1:]
    if len(itemset) != 0:
      make_tree(x , itemset)

def find_paths(A , item):
  if len(A.children) == 0:
    pass
  for i in A.children:
    if i.name == item:
      path_nodes.append(i)
    find_paths(i , item)
  return path_nodes

def list_path(pathnode):
  x = []
  while pathnode.name != 'Null':
    x.append(pathnode.name)
    pathnode = pathnode.parent
  return x

def conditional_db(pathnodes):
  all_path = []
  for i in pathnodes:
    c = []
    c.append(list_path(i))
    c.append(i.count)
    all_path.append(c)
  return all_path

def delete_by_name(l1 , item):
  flag = 0
  for i in range(len(l1)):
    if l1[i] == item:
      flag = flag + 1
      break
  if flag == 1:
    del l1[i]
  return l1


def decrease_path_length(conditional_data_base , support):
  D = {}
  for i in conditional_data_base:
    for j in i[0]:
      if j in D.keys():
        D[j] = D[j] + i[1]
      else:
        D[j] = i[1]
  items_to_be_deleted = []
  supported_items = []
  for k,v in D.items():
    if v < support:
      items_to_be_deleted.append(k)
    else:
      supported_items.append(k)
  for i in conditional_data_base:
    for j in items_to_be_deleted:
      delete_by_name(i[0] , j)
  return conditional_data_base , supported_items

def binary(number , length):
 
  ts = []
  flag = [0]*length
  count = 1
  sumi = 0
  ss = []
  for i in range(length):
    ss.append(2**(i+1))
  while sumi < number:
    sumi = 0
    for i in range(len(flag)):
      sumi = sumi + (2**(len(flag) - 1 -i))*flag[i]
    for i in range(len(ss)):
      if count%ss[i] < int((ss[i]/2)) + 1 and count%ss[i] > 0:
        flag[length- i - 1] = 0
      else:
        flag[length - i - 1] = 1
    subset = []
    for k in range(length):
      if flag[k] == 1:
        subset.append(k)
    ts.append(subset)
    count = count + 1
  return ts

def find_subsets(list1):
  length = len(list1)
  number = 2**length - 2
  sub = binary(number , length)
  sub = sub[1:]
  t_subi = []
  for i in sub:
    subi = []
    for j in i:
      subi.append(list1[j])
    t_subi.append(subi)
  return t_subi

def find_freq_items(cond_db ,unique_items, support):
  subsets = find_subsets(unique_items)
  freq_list = []
  for i in subsets:
    if len(i) > 1:
      sumi = 0
      for j in cond_db:
        if check_if_subset(i,j[0]):
          sumi = sumi + j[1]
      if sumi >= support:
        freq_list.append(i)
  return freq_list

def check_if_subset(list1 , list2):
  flag = 0
  for i in list1:
    if i in list2:
      flag = flag + 1
  if flag == len(list1):
    return True
  else:
    return False 

In [0]:
TL = []
support = 0.3
list1 = make_L1(df_as_matrix , support)
print(list1)
print("********************************")
tree_ready = prune_and_sort(df_as_matrix,list1)
tree = FP_Tree()
for i in tree_ready:
  make_tree(tree , i)
for i in range(len(list1) - 1):
  path_nodes = []
  paath = find_paths(tree, list1[i+1])
  cond = conditional_db(paath)
  xx , yy = decrease_path_length(cond , support*df_as_matrix.shape[0])
  print(find_freq_items(xx,yy,support*df_as_matrix.shape[0]))
  TL.append(find_freq_items(xx,yy,support*df_as_matrix.shape[0]))
  print("**********************************************")

[0, 'United-States', 'White', '<=50K', 'Private', 'Male', '40', 'Married-civ-spouse', 'Husband', 'Female', 'Never-married', 'HS-grad', 9]
********************************
[['United-States', 0]]
**********************************************
[['United-States', 0], ['White', 0], ['White', 'United-States'], ['White', 'United-States', 0]]
**********************************************
[['United-States', 0], ['White', 0], ['White', 'United-States'], ['White', 'United-States', 0], ['<=50K', 0], ['<=50K', 'United-States'], ['<=50K', 'United-States', 0], ['<=50K', 'White'], ['<=50K', 'White', 0], ['<=50K', 'White', 'United-States'], ['<=50K', 'White', 'United-States', 0]]
**********************************************
[['United-States', 0], ['White', 0], ['White', 'United-States'], ['White', 'United-States', 0], ['<=50K', 0], ['<=50K', 'United-States'], ['<=50K', 'United-States', 0], ['<=50K', 'White'], ['<=50K', 'White', 0], ['<=50K', 'White', 'United-States'], ['<=50K', 'White', 'United-Stat

In [0]:
import time
t0 = time.clock()
TL = []
support = 0.5
list1 = make_L1(df_as_matrix , support)
print(list1)
tree_ready = prune_and_sort(df_as_matrix,list1)
tree = FP_Tree()
for i in tree_ready:
  make_tree(tree , i)
for i in range(len(list1) - 1):
  path_nodes = []
  paath = find_paths(tree, list1[i+1])
  cond = conditional_db(paath)
  xx , yy = decrease_path_length(cond , support*df_as_matrix.shape[0])
  print(find_freq_items(xx,yy,support*df_as_matrix.shape[0]))
t1 = time.clock()
print(t1 - t0)

[0, 'United-States', 'White', '<=50K', 'Private', 'Male']
[['United-States', 0]]
[['United-States', 0], ['White', 0], ['White', 'United-States'], ['White', 'United-States', 0]]
[['United-States', 0], ['White', 0], ['White', 'United-States'], ['White', 'United-States', 0], ['<=50K', 0], ['<=50K', 'United-States'], ['<=50K', 'United-States', 0], ['<=50K', 'White'], ['<=50K', 'White', 0], ['<=50K', 'White', 'United-States'], ['<=50K', 'White', 'United-States', 0]]
[['United-States', 0], ['White', 0], ['White', 'United-States'], ['White', 'United-States', 0], ['<=50K', 0], ['Private', 0], ['Private', 'United-States'], ['Private', 'United-States', 0], ['Private', 'White'], ['Private', 'White', 0], ['Private', 'White', 'United-States'], ['Private', 'White', 'United-States', 0], ['Private', '<=50K'], ['Private', '<=50K', 0]]
[['United-States', 0], ['White', 0], ['White', 'United-States'], ['White', 'United-States', 0], ['Male', 0], ['Male', 'United-States'], ['Male', 'United-States', 0], ['Ma